In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import os
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from PIL import Image

In [2]:
## transformation grid generator##
def affine_grid_generator(height, width, theta):
    """
    This function returns a sampling grid, which when
    used with the bilinear sampler on the input feature
    map, will create an output feature map that is an
    affine transformation [1] of the input feature map.
    Input
    -----
    - height: desired height of grid/output. Used
      to downsample or upsample.
    - width: desired width of grid/output. Used
      to downsample or upsample.
    - theta: affine transform matrices of shape (num_batch, 2, 3).
      For each image in the batch, we have 6 theta parameters of
      the form (2x3) that define the affine transformation T.
    Returns
    -------
    - normalized grid (-1, 1) of shape (num_batch, 2, H, W).
      The 2nd dimension has 2 components: (x, y) which are the
      sampling points of the original image for each point in the
      target image.
    Note
    ----
    [1]: the affine transformation allows cropping, translation,
         and isotropic scaling.
    """
    num_batch = tf.shape(theta)[0]

    # create normalized 2D grid
    x = tf.linspace(-1.0, 1.0, width)
    y = tf.linspace(-1.0, 1.0, height)
    x_t, y_t = tf.meshgrid(x, y)

    # flatten
    x_t_flat = tf.reshape(x_t, [-1])
    y_t_flat = tf.reshape(y_t, [-1])

    # reshape to [x_t, y_t , 1] - (homogeneous form)
    ones = tf.ones_like(x_t_flat)
    sampling_grid = tf.stack([x_t_flat, y_t_flat, ones])

    # repeat grid num_batch times
    sampling_grid = tf.expand_dims(sampling_grid, axis=0)
    sampling_grid = tf.tile(sampling_grid, tf.stack([num_batch, 1, 1]))

    # cast to float32 (required for matmul)
    theta = tf.cast(theta, 'float32')
    sampling_grid = tf.cast(sampling_grid, 'float32')

    # transform the sampling grid - batch multiply
    batch_grids = tf.matmul(theta, sampling_grid)
    # batch grid has shape (num_batch, 2, H*W)

    # reshape to (num_batch,2, H, W)
    batch_grids = tf.reshape(batch_grids, [num_batch, 2, height, width])
    #print(batch_grids)
    return batch_grids

In [3]:
def bilinear_sampler(img, x, y):
    """
    Performs bilinear sampling of the input images according to the
    normalized coordinates provided by the sampling grid. Note that
    the sampling is done identically for each channel of the input.
    To test if the function works properly, output image should be
    identical to input image when theta is initialized to identity
    transform.
    Input
    -----
    - img: batch of images in (B, H, W, C) layout.
    - grid: x, y which is the output of affine_grid_generator.
    Returns
    -------
    - out: interpolated images according to grids. Same size as grid.
    """
    H = tf.shape(img)[1]
    W = tf.shape(img)[2]
    max_y = tf.cast(H - 1, 'int32')
    max_x = tf.cast(W - 1, 'int32')
    zero = tf.zeros([], dtype='int32')

    # rescale x and y to [0, W-1/H-1]
    x = tf.cast(x, 'float32')
    y = tf.cast(y, 'float32')
    x = 0.5 * ((x + 1.0) * tf.cast(max_x-1, 'float32'))
    y = 0.5 * ((y + 1.0) * tf.cast(max_y-1, 'float32'))

    # grab 4 nearest corner points for each (x_i, y_i)
    x0 = tf.cast(tf.floor(x), 'int32')
    x1 = x0 + 1
    y0 = tf.cast(tf.floor(y), 'int32')
    y1 = y0 + 1

    # clip to range [0, H-1/W-1] to not violate img boundaries
    x0 = tf.clip_by_value(x0, zero, max_x)
    x1 = tf.clip_by_value(x1, zero, max_x)
    y0 = tf.clip_by_value(y0, zero, max_y)
    y1 = tf.clip_by_value(y1, zero, max_y)

    # get pixel value at corner coords
    Ia = get_pixel_value(img, x0, y0)
    Ib = get_pixel_value(img, x0, y1)
    Ic = get_pixel_value(img, x1, y0)
    Id = get_pixel_value(img, x1, y1)

    # recast as float for delta calculation
    x0 = tf.cast(x0, 'float32')
    x1 = tf.cast(x1, 'float32')
    y0 = tf.cast(y0, 'float32')
    y1 = tf.cast(y1, 'float32')

    # calculate deltas
    wa = (x1-x) * (y1-y)
    wb = (x1-x) * (y-y0)
    wc = (x-x0) * (y1-y)
    wd = (x-x0) * (y-y0)

    # add dimension for addition
    wa = tf.expand_dims(wa, axis=3)
    wb = tf.expand_dims(wb, axis=3)
    wc = tf.expand_dims(wc, axis=3)
    wd = tf.expand_dims(wd, axis=3)

    # compute output
    out = tf.add_n([wa*Ia, wb*Ib, wc*Ic, wd*Id])
    return out

In [4]:
##pixel value generator##
def get_pixel_value(img, x, y):
    """
    Utility function to get pixel value for coordinate
    vectors x and y from a  4D tensor image.
    Input
    -----
    - img: tensor of shape (B, H, W, C)
    - x: flattened tensor of shape (B*H*W,)
    - y: flattened tensor of shape (B*H*W,)
    Returns
    -------
    - output: tensor of shape (B, H, W, C)
    """
    shape = tf.shape(x)
    batch_size = shape[0]
    height = shape[1]
    width = shape[2]

    batch_idx = tf.range(0, batch_size)
    batch_idx = tf.reshape(batch_idx, (batch_size, 1, 1))
    b = tf.tile(batch_idx, (1, height, width))

    indices = tf.stack([b, y, x], 3)

    return tf.gather_nd(img, indices)

In [8]:
IMGSIZE = (224, 224 ,3)
OUT_IMG_SIZE = (224 ,224 ,3)
target_size = [IMGSIZE[:2] ,OUT_IMG_SIZE[:2]]
paths = r"C:\Users\hp\anaconda\Project_files_Palmprint_detector\Test_Images"
def cos_mse(y_pred ,y_true):
    return 10*keras.losses.cosine_similarity(y_pred ,y_true) + 0.01*keras.losses.MeanSquaredError()(y_pred ,y_true)

stn_model = tf.keras.models.load_model('C:/Users/hp/anaconda/Project_files_Palmprint_detector/partial_model_step_5' , custom_objects ={'cos_mse' : cos_mse})
count = 1
img_arr = []
images_0 = os.listdir(paths)
for ind in range(len(images_0)):
    image_0 = images_0[ind]
    print(image_0)
    path_target_0 = f"{paths}\{image_0}"
    try:
        img_0 = img_to_array(load_img(path_target_0 , target_size = target_size[0])).astype(np.uint8)
        #tf.keras.preprocessing.image.array_to_img(img_0).show()
    except:
        count = 3
        print('Failed at' + path_target_0 )
        print('Retrying ...')
        while count > 0:
            try:
                img_0 = img_to_array(load_img(path_target_0 , target_size = target_size[0])).astype(np.uint8)
                print('Error mitigated . Proceeding ...')
                break
            except:
                count -= 1
                if count == 0:
                    print('Failed to mitigate error . Skipping ...')
    if count == 0:
        continue
    img_arr.append(img_0)

img_arr = np.array(img_arr)
print(img_arr.shape)

im = stn_model(img_arr)    

for i in range(len(im)):
    tf.keras.preprocessing.image.array_to_img(im[i]).show()
    #print(im[i].numpy())
    im1 = Image.fromarray(im[i].numpy().astype(np.uint8))
    im1.save(f"your_file_{i}.png")


001_1.JPG
002_1.JPG
003_2.JPG
004_1.JPG
005_1.JPG
006_1.JPG
007_1.JPG
008_2.JPG
009_1.JPG
010_1.JPG
(10, 224, 224, 3)


In [165]:
from tensorflow import keras
class sample_generator(tf.keras.utils.Sequence):
    def __init__(self , X_data ,y_data,label,batch_size):
        self.x = X_data
        self.y = y_data
        self.label = Label
        self.batch_size = batch_size
    def __len__(self) :
        return int(np.ceil(len(self.x)*(len(self.x)-1) / float(self.batch_size)))
    def __getitem__(self,idx):
        batch_x = [None ,None]
        batch_y = [[] ,[]]
        print((idx//(len(self.x)-1) + (idx%(len(self.x)-1))*self.batch_size)%(len(self.x)-1) ,(idx//(len(self.x)-1) + (idx%(len(self.x)-1))*self.batch_size)%(len(self.x)-1) + self.batch_size)
        batch_x = [np.tile(self.x[idx//(len(self.x)-1)] ,(self.batch_size ,1,1,1)) , self.y[(idx//(len(self.x)-1) + (idx%(len(self.x)-1))*self.batch_size)%(len(self.x)-1) :(idx//(len(self.x)-1) + (idx%(len(self.x)-1))*self.batch_size)%(len(self.x)-1) + self.batch_size,...]]
        batch_y[0].extend([self.label.tolist()[idx//(len(self.x)-1)]]*(self.batch_size))
        #print([self.label.tolist()[idx//(len(self.x)-1)]]*(self.batch_size) )
        batch_y[1].extend(self.label[(idx//(len(self.x)-1) + (idx%(len(self.x)-1))*self.batch_size)%(len(self.x)-1) :(idx//(len(self.x)-1) + (idx%(len(self.x)-1))*self.batch_size)%(len(self.x)-1) + self.batch_size,...].tolist())
        #print(self.label[(idx//(len(self.x)-1) + (idx%(len(self.x)-1))*self.batch_size)%(len(self.x)-1) :(((idx//len(self.x)-1)) + (idx%(len(self.x)-1))*self.batch_size)%(len(self.x)-1) + self.batch_size,...].tolist())
        batch_y = zip(batch_y[0] ,batch_y[1])
        y_arr = [0  if y[0] != y[1] else 1 for y in batch_y]
        print(len(batch_x[0]) ,len(batch_x[1]),len(y_arr))
        return batch_x , np.array(y_arr)

In [181]:
(1298//1298 + (1298%1298)*32)%1298 

1

In [104]:
((1297//1298) + (1297%1298)*32)%(1298) + 32

1298

In [8]:
import Data_Loader as DL

Code started ...
Code executed successfully


In [9]:
batch_size = 32
data_loader = DL.data_loader(Base_path = r"D:\BTP\4\IITD_Palmprint_V1")
data_loader(print_process = False  , in_dim = (IMGSIZE[0],IMGSIZE[1]) ,out_dim = (OUT_IMG_SIZE[0],OUT_IMG_SIZE[1]) )
X,y,Label,X2,y2,Label2 = data_loader.build_data()
X =X.astype('float16') #X.astype('float32')
y = y.astype('float16') # y.astype('float32')
data_loader.clear_data()

Failed atD:\BTP\4\IITD_Palmprint_V1\Right_Hand\164_5.JPG and D:\BTP\4\IITD_Palmprint_V1\Segmented\Right\164_5.bmp
Retrying ...
Failed to mitigate error . Skipping ...
Data loaded successfully with 1299 left_hand samples and 911 right_hand samples
Data_Loader is cleared .


In [93]:
Label

array(['001', '001', '001', ..., '230', '230', '230'], dtype='<U3')

In [94]:
len(X) - 1

1298

In [105]:
y.shape

(1299, 224, 224, 3)

In [166]:
generator = sample_generator(X , y,Label,batch_size)

In [61]:
len(generator[0])

2

In [62]:
len(generator[0][0])

2

In [125]:
generator[0][1].shape

(32,)

In [196]:
generator[2596][1]

2 34
32 32 32


array([1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [133]:
generator[1][0][1].shape

(31, 224, 224, 3)

In [197]:
tf.keras.preprocessing.image.array_to_img(generator[1298][0][1][20]).show()

1 33
32 32 32
